Observations and Insights:

# Toolkit and File Setup

In [49]:
# Import Dependencies and Modules
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st


# Load Files
mouse_metadata_path = "/Users/brianroberts1/Documents/GitHub/matplotlib_challenge/Mouse_metadata.csv"
study_results_path = "/Users/brianroberts1/Documents/GitHub/matplotlib_challenge/Study_results.csv"

# Read the Mouse Data and Study Results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_and_study = pd.merge(mouse_metadata, study_results,on = ["Mouse ID"])

# Display the data table for preview
mouse_and_study

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


# Data Cleaning

In [50]:
# Checking the number of mice for duplicates. 

# Based on the study_results, each mouse should have 10 entries,...
# ... any more should be considered a recording error and removed.  
mouse_and_study['Mouse ID'].value_counts().head()


g989    13
v991    10
y793    10
a262    10
c758    10
Name: Mouse ID, dtype: int64

In [51]:
### Remove data for Mouse ID: g989, as it has more than 10 entries ###

# Filter the rows that contain 'g989' and store the indexes
drop_ids = mouse_and_study[mouse_and_study['Mouse ID'] == 'g989'].index

# Delete matching row indexes from the dataframe
mouse_and_study.drop(drop_ids, inplace = True)

# Check cleaned dataframe
mouse_and_study['Mouse ID'].value_counts()


v991    10
y163    10
m650    10
j984    10
k403    10
        ..
u153     1
d133     1
h428     1
l872     1
f932     1
Name: Mouse ID, Length: 248, dtype: int64

# Summary Statistics

In [73]:
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
#--------------------------------------------------------------------------------------------------------

# Create a dataframe for the mean of tumor growth for each drug regimen.
tumor_means = pd.DataFrame(mouse_and_study.groupby(['Drug Regimen','Timepoint'])['Tumor Volume (mm3)'].mean())

# Create a dataframe for the median of tumor growth for each drug regimen.
tumor_medians = pd.DataFrame(mouse_and_study.groupby(['Drug Regimen','Timepoint'])['Tumor Volume (mm3)'].median())

# Create a dataframe for the variance of tumor growth for each drug regimen.
tumor_vars = pd.DataFrame(mouse_and_study.groupby(['Drug Regimen','Timepoint'])['Tumor Volume (mm3)'].var())

# Create a dataframe for the standard deviation of tumor growth for each drug regimen.
tumor_stds = pd.DataFrame(mouse_and_study.groupby(['Drug Regimen','Timepoint'])['Tumor Volume (mm3)'].std())

# Create a dataframe for the SEM of tumor growth for each drug regimen.
tumor_sems = pd.DataFrame(mouse_and_study.groupby(['Drug Regimen','Timepoint'])['Tumor Volume (mm3)'].sem())


# Assemble the resulting series into a single summary dataframe.
#---------------------------------------------------------------
tumor_summary = pd.(tumor_means, tumor_medians, tumor_vars, tumor_stds, tumor_sems)
tumor_summary


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line

# Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.


In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot


# Quartiles, Outliers, and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


# Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


# Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
